<a href="https://colab.research.google.com/github/Imbana/ClasificacionLimonesCNN/blob/editar/Codigo_proyecto_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

---
# Clasificacion de limones Tahiti con redes neuronales convolucionales (CCN)
---
se recomienda configurar el enterno de ejecucion con TPU(Una unidad de procesamiento tensorial) sirve como acelerador de IA desarrollado por Google para el aprendizaje automático con redes neuronales artificiales y más específicamente optimizado para usar TensorFlow. entorno de ejecucion>>cambiar tipo de entorno de ejecucion>>TPU


CONFIGURACION DEL ENTORNO Y LIBRERIAS

In [ ]:
#se conecta con google drive donde se encuentran las carpetas con las imagenes de los limones

from google.colab import drive
drive.mount('/content/drive/')


Mounted at /content/drive/


In [ ]:
#librerias

import matplotlib.pylab as plt
import tensorflow as tf
import numpy as np

import os

import cv2


import PIL
import glob
import shutil
from PIL import Image

import keras_preprocessing
from keras_preprocessing import image
from keras_preprocessing.image import ImageDataGenerator


from tensorflow import keras

#from tensorflow.keras import models

from tensorflow import lite

---
#Generar el dataset
---
Se copia la ruta de la carpeta donde estan las imagenes y se realiza una generacion de dataset con las caracteristicas necesarias como tamaño de imagen, escala, rotacion, tamaño de batch etc.

PREPOCESAMIENTO DE LAS IMAGENES

Para este paso se necesita carpetas de imagenes con las categorias de limones, o lo que se quiera clasificar, en nuestro caso 3 carpetas diferentes con imagenes de limones con Acaros, Buenos y con manchas


Antes de crear los arrays que entran a la red neuronal se necesita hacerle un preprocesamiento a estas carpetas o datos, se necesita dividir en tres tipo de datos, los de entrenamiento y los de validacion (test y para prueba), se necesita que las imagenes sean aleatorias, y que cada carpeta poseea la misma cantidad de imagenes para no sobresaturar un clase, ademas de que podemos aumetar el dataset con modificaciones a las imagenes modificaciones como de rotacion, modificacion de entorno, color etc.


In [ ]:
# crear las carpetas, se necesita la carpeta Train, Test y prueba. dentro de esta 3 carpetas  para las clases de limones


import os
import glob
import shutil
import random

# la primera ruta es la direccion donde se va a guardar los datos
#new_folder_path = os.path.join('/content/drive/My Drive/Proyecto_grado/Fotos/Datos4Clases')

new_folder_path = os.path.join('/DatosPrueba')

Data_train=os.path.join(new_folder_path,'Datos_Train')
Data_test=os.path.join(new_folder_path,'Datos_Test')
Data_prueba=os.path.join(new_folder_path,'Datos_Prueba')

if not os.path.exists(new_folder_path): # Check la existencia del directorio
    os.mkdir(new_folder_path)
    os.mkdir(Data_train)
    os.mkdir(Data_test)
    os.mkdir(Data_prueba)
    #os.mkdir(os.path.join(Data_train,"Maduros"))
    os.mkdir(os.path.join(Data_train,"Acaros"))
    os.mkdir(os.path.join(Data_train,"Buenos"))
    os.mkdir(os.path.join(Data_train,"Manchas"))
    #os.mkdir(os.path.join(Data_test,"Maduros"))
    os.mkdir(os.path.join(Data_test,"Acaros"))
    os.mkdir(os.path.join(Data_test,"Buenos"))
    os.mkdir(os.path.join(Data_test,"Manchas"))
    #os.mkdir(os.path.join(Data_prueba,"Maduros"))
    os.mkdir(os.path.join(Data_prueba,"Acaros"))
    os.mkdir(os.path.join(Data_prueba,"Buenos"))
    os.mkdir(os.path.join(Data_prueba,"Manchas"))





# Copia de datos de /Fotos/Maduros a /Fotos/Data/Maduros version 2.0

#ruta donde se encuentran los datos originales
parent_dir = '/content/drive/My Drive/Proyecto_grado/Fotos/Raw-Data'



################################################################################
# Lista los datos de la carpeta y los pone aleatoriamente  (desordena las imagenes)
f_b = glob.glob(os.path.join(parent_dir,'Buenos','*.jpg'))
random.shuffle(f_b)

#f_m = glob.glob(os.path.join(parent_dir,'Maduros','*.jpg')) 
#random.shuffle(f_m)

f_a = glob.glob(os.path.join(parent_dir,'Acaros','*.jpg'))  
random.shuffle(f_a)

f_s = glob.glob(os.path.join(parent_dir,'Manchas','*.jpg'))  
random.shuffle(f_s)


##################################################
# ver la carpeta con menos datos 4 clases

#valores=[len(f_b),len(f_m),len(f_a),len(f_s)]


valores=[len(f_b),len(f_a),len(f_s)] # para tres clases

print(min(valores))


print('Buenos: {}'.format(len(f_b)))
#print('Maduros: {}'.format(len(f_m)))
print('Acaros: {}'.format(len(f_a)))
print('Manchas: {}'.format(len(f_s)))



################################################################################

# pegar las imagenes en las nuevas carpetas

limit = min(valores)

limite70=int(limit*0.8)
limite20=int(limit*0.15)
limite10=int(limit*0.05)

i = 0
for f in range(limite70):
  # nombre de la imagen
  name = 'train' + str(i) + '.jpg'
  
  n1 = 'h' + str(i) + '.jpg' # New name for the images flip horizontal
  n2 = 'v' + str(i) + '.jpg' # New name for the images flip vertical
#  n3 = 'r' + str(i) + '.jpg' # New name for the images rotated


#pegar una imagen en un nuevo path con un nuevo nombre (Train)

  #im = Image.open(f_s[f])
  #i1=im.transpose(PIL.Image.FLIP_LEFT_RIGHT)
  #i1.save(os.path.join(os.path.join(Data_train, 'Manchas'), n1), quality=95)
  #i2 = im.transpose(PIL.Image.FLIP_TOP_BOTTOM)
  #i2.save(os.path.join(os.path.join(Data_train, 'Manchas'), n2), quality=95)
  shutil.copy(f_s[f], os.path.join(os.path.join(Data_train, 'Manchas'), name))

  #im = Image.open(f_b[f])
  #i1=im.transpose(PIL.Image.FLIP_LEFT_RIGHT)
  #i1.save(os.path.join(os.path.join(Data_train, 'Buenos'), n1), quality=95)
  #i2 = im.transpose(PIL.Image.FLIP_TOP_BOTTOM)
  #i2.save(os.path.join(os.path.join(Data_train, 'Buenos'), n2), quality=95)
  shutil.copy(f_b[f], os.path.join(os.path.join(Data_train, 'Buenos'), name))

  #shutil.copy(f_m[f], os.path.join(os.path.join(Data_train, 'Maduros'), name))
  #im = Image.open(f_a[f])
  #i1=im.transpose(PIL.Image.FLIP_LEFT_RIGHT)
  #i1.save(os.path.join(os.path.join(Data_train, 'Acaros'), n1), quality=95)
  #i2 = im.transpose(PIL.Image.FLIP_TOP_BOTTOM)
  #i2.save(os.path.join(os.path.join(Data_train, 'Acaros'), n2), quality=95)
  shutil.copy(f_a[f], os.path.join(os.path.join(Data_train, 'Acaros'), name))
  
  i+=1

#pegar una imagen en un nuevo path con un nuevo nombre (Test)
i = 0
for f in range(limite70,limite70+limite20):

  name = 'test' + str(i) + '.jpg'

  n1 = 'h' + str(i) + '.jpg' # New name for the images flip horizontal
  n2 = 'v' + str(i) + '.jpg' # New name for the images flip vertical
#  n3 = 'r' + str(i) + '.jpg' # New name for the images rotated


#pegar una imagen en un nuevo path con un nuevo nombre (Train)

  #im = Image.open(f_s[f])
  #i1=im.transpose(PIL.Image.FLIP_LEFT_RIGHT)
  #i1.save(os.path.join(os.path.join(Data_test, 'Manchas'), n1), quality=95)
  #i2 = im.transpose(PIL.Image.FLIP_TOP_BOTTOM)
  #i2.save(os.path.join(os.path.join(Data_test, 'Manchas'), n2), quality=95)
  shutil.copy(f_s[f], os.path.join(os.path.join(Data_test, 'Manchas'), name))

  #im = Image.open(f_b[f])
  #i1=im.transpose(PIL.Image.FLIP_LEFT_RIGHT)
  #i1.save(os.path.join(os.path.join(Data_test, 'Buenos'), n1), quality=95)
  #i2 = im.transpose(PIL.Image.FLIP_TOP_BOTTOM)
  #i2.save(os.path.join(os.path.join(Data_test, 'Buenos'), n2), quality=95)
  shutil.copy(f_b[f], os.path.join(os.path.join(Data_test, 'Buenos'), name))

  #shutil.copy(f_m[f], os.path.join(os.path.join(Data_train, 'Maduros'), name))
  #im = Image.open(f_a[f])
  #i1=im.transpose(PIL.Image.FLIP_LEFT_RIGHT)
  #i1.save(os.path.join(os.path.join(Data_test, 'Acaros'), n1), quality=95)
  #i2 = im.transpose(PIL.Image.FLIP_TOP_BOTTOM)
  #i2.save(os.path.join(os.path.join(Data_test, 'Acaros'), n2), quality=95)
  shutil.copy(f_a[f], os.path.join(os.path.join(Data_test, 'Acaros'), name))
  
  i+=1


#pegar una imagen en un nuevo path con un nuevo nombre (Prueba)
i = 0
for f in range(limite70+limite20,limite70+limite20+limite10):

  name = 'prueba' + str(i) + '.jpg'
  #n3 = 'r' + str(i) + '.jpg' # New name for the images rotated

  shutil.copy(f_s[f], os.path.join(os.path.join(Data_prueba, 'Manchas'), name))
  shutil.copy(f_b[f], os.path.join(os.path.join(Data_prueba, 'Buenos'), name))
  #shutil.copy(f_m[f], os.path.join(os.path.join(Data_prueba, 'Maduros'), name))
  shutil.copy(f_a[f], os.path.join(os.path.join(Data_prueba, 'Acaros'), name))
  
  i+=1

In [ ]:
# mirar cuantas imagenes tiene cada carpeta

parent_dir = '/content/drive/My Drive/Proyecto_grado/Fotos/Raw-Data'


f_b = glob.glob(os.path.join(parent_dir,'Buenos','*.jpg'))

#f_m = glob.glob(os.path.join(parent_dir,'Maduros','*.jpg')) 

f_a = glob.glob(os.path.join(parent_dir,'Acaros','*.jpg'))  

f_s = glob.glob(os.path.join(parent_dir,'Manchas','*.jpg'))  


print('Buenos: {}'.format(len(f_b)))
#print('Maduros: {}'.format(len(f_m)))
print('Acaros: {}'.format(len(f_a)))
print('Manchas: {}'.format(len(f_s)))

Buenos: 1295
Acaros: 642
Manchas: 758


In [ ]:
#enpaquetar zip o inlcuso otros formatos

import shutil

archivo_zip = shutil.make_archive("/content/drive/My Drive/Dataset/Dataset3Clases", "zip", "/DatosPrueba")


In [ ]:
%ls
%cd /.. #llevar el directorio a la carpeta
%ls

In [ ]:
# Archivo a copiar    #Path donde se va a copiar solo copiar

%cp -av /DatosPrueba /content/drive/My\ Drive/Dataset/Datos3Clases

In [ ]:



TRAINING_DIR = "/DatosPrueba/Datos_Train" #Data_train

training_datagen = ImageDataGenerator(
      rescale = 1./255,
	    rotation_range=40,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest')


VALIDATION_DIR = "/DatosPrueba/Datos_Test" #Data_test
validation_datagen = ImageDataGenerator(rescale = 1./255)

train_generator = training_datagen.flow_from_directory(
	TRAINING_DIR,
	target_size=(224,224),
	class_mode='categorical',
  shuffle=True,
  batch_size=32
)

validation_generator = validation_datagen.flow_from_directory(
	VALIDATION_DIR,
	target_size=(224,224),
	class_mode='categorical',
  shuffle=True,
  batch_size=32
)


#visualizar informacion del dataset

image_batch_train, label_batch_train = next(iter(train_generator))

print("composicion de un batch: ", image_batch_train.shape)
print("composicion de la etiqueta: ", label_batch_train.shape)
print(train_generator.class_indices)


In [ ]:
np.save('xtrain.npy',validation_generator)

In [ ]:
print(train_generator)
print(validation_generator.n)
print(train_generator.batch_size)
print(train_generator.samples)
print(validation_generator.batch_size)




---
#Configuracion del modelo de red neuronal
---
se extrae de la libreria de keras, luego de modifica sus capas finales para clasificar 3 tipos de clases 

```
# Tiene
```



In [ ]:
#un modelo creado desde cero
"""model = tf.keras.models.Sequential([
    # Note the input shape is the desired size of the image 150x150 with 3 bytes color
    # This is the first convolution
    
    tf.keras.layers.Conv2D(64, (3,3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    # The second convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # The third convolution
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # The fourth convolution
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    
    # Flatten the results to feed into a DNN
    # 
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.5),
    # 512 neuron hidden layer
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(3, activation='softmax')
])"""

In [ ]:
# se extrae un modelo base con sus pesos entrenados(weights='imagenet') y sin incluir las ultimas capas(include_top=False) luego se modifica agregando las ultimas capas
def modelo1():

  base_model = tf.keras.applications.MobileNetV2(input_shape=(224,224,3),
                                                include_top=False, 
                                                weights='imagenet')
  base_model.trainable = False
  model = tf.keras.Sequential([
    base_model,
    tf.keras.layers.Conv2D(32, 3, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(3, activation='softmax')
  ])

  return model



In [ ]:
#Creacion de otro modelo con menos capas finales ## mobile = keras.applications.mobilenet.MobileNet()


def modelo2():
    modeloV2 = tf.keras.applications.MobileNetV2(input_shape=(224,224,3),include_top=False, weights='imagenet')
    modeloV2.trainable = False
    model = tf.keras.Sequential([modeloV2,
                          keras.layers.GlobalAveragePooling2D(),
                          keras.layers.Dense(3, activation='softmax')])
    return model

In [ ]:
#mobile=modelo1()
#mobile.summary()
base_model = tf.keras.applications.MobileNet(input_shape=(224,224,3),
                                                include_top=False, 
                                                weights='imagenet')

base_model.summary()

---
##Entrenamiento del modelo
---
se especifica la configuracion del entrenamiento como tipo de optimizador de gradiente, funciin de costo, cuantas epocas  etc.

In [ ]:
#llama modelo
model=modelo1()

# configuro el entrenamiento de la red
model.compile(optimizer='Adam',loss='categorical_crossentropy',metrics=['accuracy'])


# valores para pasos en epocas
steps_per_epoch = np.ceil(train_generator.samples/train_generator.batch_size)
val_steps_per_epoch = np.ceil(validation_generator.samples/validation_generator.batch_size)

# entrenamiento de la red neuronal
history=model.fit(
    
    train_generator,
    steps_per_epoch=18,
    epochs=25,
    verbose = 1,
    validation_data = validation_generator,
    validation_steps=val_steps_per_epoch
    
    )

---
Pruebas del modelo o test del modelo entrenado

---




In [ ]:
# Evaluar en el modelo 

final_loss, final_accuracy = model.evaluate(validation_generator, steps =5)

print("Final loss: {:.2f}".format(final_loss))
print("Final accuracy: {:.2f}%".format(final_accuracy * 100))

#mobileNETV2 primero

In [ ]:
#####

#grafica del entrenamiento y la validacion

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))
plt.plot(epochs, acc, 'r', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend(loc=0)

params = {"text.color" : "red",
          "xtick.color" : "white",
          "ytick.color" : "white"}
plt.rcParams.update(params)
plt.savefig("/content/Resultado.jpg")

plt.show()

In [ ]:
#prediccion de una imagen especifica

# funion para preparar  una imagen para pasarla por el modelo entrenado
Ruta=('/DatosPrueba/Datos_Prueba/Manchas/prueba15.jpg')
img=plt.imread(Ruta)

plt.imshow(img)
plt.show()


def prepare_image(file):
    img_path = ''
    img = image.load_img(img_path + file, target_size=(224, 224))
    img_array = image.img_to_array(img) # convertir en  array
    img_array_expanded_dims = np.expand_dims(img_array, axis=0) # agregar el cero
    
    return keras.applications.mobilenet.preprocess_input(img_array_expanded_dims)

#prediccion
preprocessed_image = prepare_image(Ruta)
predictions = model.predict(preprocessed_image)
print(predictions)
print(train_generator.class_indices)


In [ ]:
dataset_labels = sorted(train_generator.class_indices.items(), key=lambda pair:pair[1])
dataset_labels = np.array([key.title() for key, value in dataset_labels])
print(dataset_labels)

In [ ]:
import pandas as pd
val_image_batch, val_label_batch = next(iter(validation_generator))
true_label_ids = np.argmax(val_label_batch, axis=-1)

print("Validation batch shape:", val_image_batch.shape)


tf_model_predictions = model(val_image_batch)
print("Prediction results shape:", tf_model_predictions.shape)


tf_pred_dataframe = pd.DataFrame(tf_model_predictions.numpy())
tf_pred_dataframe.columns = dataset_labels

print("Prediction results for the first elements")
tf_pred_dataframe.head()

In [ ]:
predicted_ids = np.argmax(tf_model_predictions, axis=-1)
predicted_labels = dataset_labels[predicted_ids]


#Print images batch and labels predictions

plt.figure(figsize=(10,15))
plt.subplots_adjust(hspace=0.4)
for n in range(30):
  plt.subplot(6,5,n+1)
  plt.imshow(val_image_batch[n])
  color = "green" if predicted_ids[n] == true_label_ids[n] else "red"
  plt.title(predicted_labels[n].title(), color=color)
  plt.axis('off')
_ = plt.suptitle("Model predictions (green: correct, red: incorrect)")

plt.savefig("/content/ImagenelimonesPrueba.jpg")



---

# EXPORTAR TU MODELO A TFLITE


---

el modelo entrenado anteriormente esta en tensorflow ahora se buscar convertido a tensorflow lite para correrlo en la raspberry.

se debe de crear tanto el modelo como las equites de salida para ver en que posicion esta cada categoria de limon(maduro, manchas, acaros, bueno)

In [ ]:
converter = lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

open("modelofinal.tflite", 'wb').write(tflite_model)


etiquetas=train_generator.class_indices
file = open("/content/prueba.txt", "w")
for llaves in etiquetas.keys():
  file.write(llaves + os.linesep)
file.close()


In [ ]:
print("Number of layers in the base model: ", len(model.layers))

print('Number of trainable variables = {}'.format(len(model.trainable_variables)))


---
# AXELERATE
---


In [ ]:

#%tensorflow_version 1.x
#we need imgaug 0.4 for image augmentations to work properly, see https://stackoverflow.com/questions/62580797/in-colab-doing-image-data-augmentation-with-imgaug-is-not-working-as-intended
!pip uninstall -y imgaug && pip uninstall -y albumentations && pip install imgaug==0.4
!git clone https://github.com/AIWintermuteAI/aXeleRate.git



import sys
sys.path.append('/content/aXeleRate')
from axelerate import setup_training,setup_inference



Uninstalling imgaug-0.4.0:
  Successfully uninstalled imgaug-0.4.0
  Using cached https://files.pythonhosted.org/packages/66/b1/af3142c4a85cba6da9f4ebb5ff4e21e2616309552caca5e8acefe9840622/imgaug-0.4.0-py2.py3-none-any.whl
fatal: destination path 'aXeleRate' already exists and is not an empty directory.


AttributeError: ignored

In [ ]:
TRAINING_DIR=("/DatosPrueba1/Datos_Train")
VALIDATION_DIR=("/DatosPrueba1/Datos_Test")
config = {
    "model" : {
        "type":                 "Classifier",
        "architecture":         "MobileNet1_0",
        "input_size":           224,
        "fully-connected":      [],
        "labels":               [],
        "dropout" : 		0.2
    },
     "weights" : {
            "full":   				"",
            "backend":   		    "imagenet",
            "save_bottleneck":      False
        
    },
    "train" : {
        "actual_epoch":         25,
        "train_image_folder":   TRAINING_DIR,
        "train_times":          4,
        "valid_image_folder":   VALIDATION_DIR,
        "valid_times":          4,
        "valid_metric":         "val_accuracy",
        "batch_size":           32,
        "learning_rate":        1e-3,
        "saved_folder":   		F"/content/drive/My Drive/Proyecto_grado/Fotos",
        "first_trainable_layer": "",
        "augumentation":				True
    },
    "converter" : {
        "type":   				["k210","tflite"]
    }
}

In [ ]:
from keras import backend as K 
K.clear_session()
model_path = setup_training(config_dict=config)

